In [ ]:
# loss 1.34, trained on A100 40GB in about 10 minutes in 4153 iterations

# --------------- train -----------------
import torch
import torch.nn.functional as F

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda' and torch.cuda.is_bf16_supported():
    dtype = torch.bfloat16
elif device == 'cuda':
    dtype = torch.float16
else:
    dtype = torch.float32

print(f"Using: {device}, dtype: {dtype}")

# config
lr = 5e-4
emb_vec_len = 256 # emb vector length
n_transformer_blocks = 16 # number of transformer blocks
mlp_dim = 128 # hidden layer size in MLP
ctx_len = 128 # max number of tokens model looks at in one go including itself, some may be zeroed out in causal attention
# batch_size = 4096 # for 40GB A100
batch_size = 2 # number of sequences per batch

# load data
with open("input.txt", "r") as f:
    text = f.read()

# char tokenizer
chars = sorted(set(text))
vocab_size = len(chars) # how many unique tokens (characters for this version)
stoi = {c:i for i,c in enumerate(chars)} # makes dict for token to int id
itos = {i:c for c,i in stoi.items()} # makes dict for id_int to token_char
encode = lambda s: [stoi[c] for c in s] # encoder: take string, output list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take list of integers, output string
print(f"Vocab size: {vocab_size}, emb_vec_len: {emb_vec_len}, transformer blocks: {n_transformer_blocks}")

# weights
emb = torch.randn(vocab_size, emb_vec_len, device=device, dtype=dtype) * 0.02 # matrix of emb vectors (picture each row is an emb vector)
pos = torch.randn(ctx_len, emb_vec_len, device=device, dtype=dtype) * 0.02 # matrix of pos emb vectors (picture 1d... pos like 3 says you start here, that 1D emb element like 5 added tells where it ends, like 5+3=8, and positive eg tells what direction. does for each dim so nudges the vector in specifid directions.)

layers = []
for _ in range(n_transformer_blocks):
    layer = {
        'wq': torch.randn(emb_vec_len, emb_vec_len, device=device, dtype=dtype) * (0.1 / (n_transformer_blocks ** 0.5)),
        'wk': torch.randn(emb_vec_len, emb_vec_len, device=device, dtype=dtype) * (0.1 / (n_transformer_blocks ** 0.5)),
        'wv': torch.randn(emb_vec_len, emb_vec_len, device=device, dtype=dtype) * (0.1 / (n_transformer_blocks ** 0.5)),
        'w1': torch.randn(emb_vec_len, mlp_dim, device=device, dtype=dtype) * (0.1 / (n_transformer_blocks ** 0.5)),
        'w2': torch.randn(mlp_dim, emb_vec_len, device=device, dtype=dtype) * (0.1 / (n_transformer_blocks ** 0.5)),
    }
    layers.append(layer)

params = [emb, pos] + [w for layer in layers for w in layer.values()]
for p in params:
    p.requires_grad = True

def rmsnorm(x, eps=1e-5):
    return x / ((x ** 2).mean(dim=-1, keepdim=True).sqrt() + eps) # denominator is like standar deviation except it subtracts 0 instead of mean

def forward(x): # is a matrix of token ids. x is batch of 1d loodup id's, aka x is 2d, but just break it down and think in terms of 1d because the other d is just for parallelization
    B, T = x.shape # B = batch size, T = time = sequence length (number of tokens in each sequence in the batch, = ctx_len - 1, because it predicts the next token)
    x = emb[x] + pos[:T] # this is where pos vector is added to emb vector... kind of like adding b in y = wx + b... x = id's, 1 id grabs 1 emb vector from the emb matrix ... 

    for layer in layers:
        nx = rmsnorm(x) # scales x
        q, k, v = nx @ layer['wq'], nx @ layer['wk'], nx @ layer['wv']
        scale = q.shape[-1] ** 0.5 # scales scores
        scores = q @ k.transpose(-2, -1) / scale
        
        # triangular causal mask
        rows = torch.arange(T, device=device).unsqueeze(1) # (T, 1)
        cols = torch.arange(T, device=device).unsqueeze(0) # (1, T)
        mask = cols <= rows # True where col <= row (lower triangle)
        scores = torch.where(mask, scores, torch.tensor(float('-inf'), device=device)) # keep score if mask=True, else -inf so softmax gives 0
        
        # print(f'rows: {rows.shape}, cols: {cols.shape}, mask: {mask.shape}, scores: {scores.shape}, T: {T}')

        x = x + F.softmax(scores, dim=-1) @ v
        x = x + (rmsnorm(x) @ layer['w1']).relu() @ layer['w2']

    return x @ emb.T

if hasattr(torch, 'compile'):
    forward = torch.compile(forward)
    print("Using torch.compile")

tokens = torch.tensor(encode(text), device=device)
all_seqs = tokens.unfold(0, ctx_len, 1)

# train
opt = torch.optim.Adam(params, lr=lr, fused=True) # optimizer = update weights eg.: w-= lr*gradient

for i in range(10000):
    idx = torch.randint(0, all_seqs.size(0), (batch_size,))
    x, y = all_seqs[idx, :-1], all_seqs[idx, 1:]

    logits = forward(x)
    loss = F.cross_entropy(logits.view(-1, vocab_size), y.view(-1))

    opt.zero_grad()
    loss.backward()
    opt.step()

    if i % 1 == 0:
        print(f"{i}: loss={loss.item():.2f}")

'''
loss = 1.34
# config
lr = 5e-4
emb_vec_len = 256 # emb vector length
n_transformer_blocks = 16 # number of transformer blocks
mlp_dim = 128 # hidden layer size in MLP
ctx_len = 128 # max number of tokens model looks at in one go including itself, some may be zeroed out in causal attention
batch_size = 4096 # number of sequences per batch

Eg. of output from prompt 'First Citizens':

First Citizens of found him appearl with me intood
not the lifter removetle like a storm'd;
And fellong where tha
'''

--2026-02-06 13:39:03--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  2.69MB/s    in 0.4s    

2026-02-06 13:39:04 (2.69 MB/s) - ‘input.txt.2’ saved [1115394/1115394]

Using: cpu, dtype: torch.float32
Vocab size: 65, emb_vec_len: 256, transformer blocks: 16
Using torch.compile
0: loss=4.25
1: loss=4.07
2: loss=4.27


KeyboardInterrupt: 

In [3]:
# -------------- inference generate -----------------
ctx = "First Citizen"
tokens = encode(ctx)

for _ in range(100):
    seq = tokens[-(ctx_len-1):]
    pad_len = (ctx_len - 1) - len(seq)
    x = torch.tensor([([0] * pad_len) + seq], device=device)
    logits = forward(x)
    probs = F.softmax(logits[0, -1] / 0.8, dim=-1)
    next_token = torch.multinomial(probs, 1).item()
    tokens.append(next_token)

print(decode(tokens))

First Citizen
eiirseAi'Al
kiy
;iBI
Kvee
dBe.riT$iiAw
yApmAmr,cRemxekTd
g3ecRRRT&oeiTbekwd:iM
o-m

ieewAo

gzeieet
